In [197]:
import os
import sys
import string
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

In [2]:
!cat aclImdb/train/pos/4077_10.txt

I first saw this back in the early 90s on UK TV, i did like it then but i missed the chance to tape it, many years passed but the film always stuck with me and i lost hope of seeing it TV again, the main thing that stuck with me was the end, the hole castle part really touched me, its easy to watch, has a great story, great music, the list goes on and on, its OK me saying how good it is but everyone will take there own best bits away with them once they have seen it, yes the animation is top notch and beautiful to watch, it does show its age in a very few parts but that has now become part of it beauty, i am so glad it has came out on DVD as it is one of my top 10 films of all time. Buy it or rent it just see it, best viewing is at night alone with drink and food in reach so you don't have to stop the film.<br /><br />Enjoy

In [4]:
len(os.listdir("aclImdb/train/neg"))

12500

In [6]:
os.listdir("aclImdb/train/neg")[0]

'1821_4.txt'

In [3]:
DATA_DIR = 'aclImdb'

In [131]:
def make_dataset_(data_file):
    with open (os.path.join(data_file), 'r') as f:
        data = f.read()
        return data

pos_dataset_ = []
neg_dataset_ = []

for data_file in os.listdir(os.path.join(DATA_DIR, 'train', 'pos')):
    data_path = os.path.join(DATA_DIR, 'train', 'pos', data_file)
    pos_dataset_.append(make_dataset_(data_path))

for data_file in os.listdir(os.path.join(DATA_DIR, 'train', 'neg')):
    data_path = os.path.join(DATA_DIR, 'train', 'neg', data_file)
    neg_dataset_.append(make_dataset_(data_path))

dataset_ = pos_dataset_ + neg_dataset_

In [132]:
print(len(pos_dataset_))
print(len(neg_dataset_))

12500
12500


In [134]:
pos_dataset_[0]

'For a movie that gets no respect there sure are a lot of memorable quotes listed for this gem. Imagine a movie where Joe Piscopo is actually funny! Maureen Stapleton is a scene stealer. The Moroni character is an absolute scream. Watch for Alan "The Skipper" Hale jr. as a police Sgt.'

In [35]:
def standardize(text):
    text = text.lower()
    return "".join(char for char in text if char
                   not in string.punctuation)

In [36]:
def tokenize(text):
    text = standardize(text)
    return text.split()

In [37]:
standardize('BOok#022')

'book022'

In [39]:
tokenize('BOok#02 Is something good?!')

['book02', 'is', 'something', 'good']

In [129]:
class Vocabulary():
    def __init__(self, freq=3):

        self.stoi = {'<pad>':0 ,'<SOS>':1, '<EOS>':2, '<UNK>':3}
        self.itos = {0:'<pad>', 1:'<SOS>', 2:'<EOS>', 3:'<UNK>'}

        self.freq = freq

    def standardize(text):
        text = text.lower()
        return "".join(char for char in text if
                    char not in string.punctuation)
    
    def tokenize(text):
            text = standardize(text)
            return text.split()

    def make_tokens(self, dataset):
        temp_vocab = {}

        for text in dataset:
            tokens = tokenize(text)

            for token in tokens:
                if token not in temp_vocab:
                    temp_vocab[token] = 1
                else:
                    temp_vocab[token] +=1

                if temp_vocab[token] == self.freq:
                    index = len(self.stoi)
                    self.stoi[token] = index
                    self.itos[index] = token
            
    def encode(self, text):
        tokens = tokenize(text)
        return ([self.stoi['<SOS>']] + [self.stoi.get(token, 3) for token in tokens]
                 + [self.stoi['<EOS>']])

    def decode(self, int_seq):
        return " ".join(self.itos.get(int, 'UNK') for int in int_seq)     

In [137]:
tokenizer_ = Vocabulary(3)
tokenizer_.make_tokens(dataset_)
encoded_ = tokenizer_.encode(dataset_[0])
decoded_ = tokenizer_.decode(encoded)

In [138]:
decoded_

'<SOS> for a movie that gets no respect there sure are a lot of memorable quotes listed for this gem imagine a movie where joe piscopo is actually funny maureen stapleton is a scene stealer the moroni character is an absolute scream watch for alan the skipper hale jr as a police sgt <EOS>'

In [139]:
type(decoded_)

str

In [103]:
from glob import glob
import random

In [124]:
def build_vocab(data_dir : str, tokenizer: Vocabulary) -> Vocabulary:

    dataset = []
    path_list = []

    path_ = os.path.join(data_dir, 'train')
    path_list = path_list = (glob(os.path.join(path_, 'pos', '*.txt')) +
                     glob(os.path.join(path_, 'neg', '*.txt')))

    for data_path in path_list:
        with open(data_path, 'r') as f:
            text = f.read()
            dataset.append(text)

    tokenizer.make_tokens(dataset)
    return tokenizer

In [117]:
class IMDBDataset(Dataset):
    def __init__(self, data_dir, tokenizer=Vocabulary, train=True, random_state=42):
        super().__init__()

        self.data_dir = data_dir
        self.nlp = tokenizer
        self.data = []

        if train:
            path = os.path.join(data_dir, 'train')
        else:
            path = os.path.join(data_dir, 'test')
        
        for label in ['pos', 'neg']:
            final_path = os.path.join(path, label)
            for i in glob(final_path + '/*.txt'):
                self.data.append((i, label == 'pos'))

        random.Random(random_state).shuffle(self.data)

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):

        file_path, label = self.data[idx]
        
        with open(file_path, 'r') as f:
            text_data = f.read()
            data = self.nlp.encode(text_data)
        return torch.tensor(data).long(), torch.tensor([label])

In [140]:
DATA_DIR = 'aclImdb'

tokenizer = Vocabulary(freq = 3)

nlp = build_vocab(DATA_DIR, tokenizer)

In [142]:
nlp.stoi

{'<pad>': 0,
 '<SOS>': 1,
 '<EOS>': 2,
 '<UNK>': 3,
 'a': 4,
 'is': 5,
 'for': 6,
 'movie': 7,
 'the': 8,
 'with': 9,
 'as': 10,
 'but': 11,
 'raines': 12,
 'by': 13,
 'are': 14,
 'of': 15,
 'and': 16,
 'to': 17,
 'i': 18,
 'was': 19,
 'who': 20,
 'that': 21,
 'ring': 22,
 'br': 23,
 'joe': 24,
 'his': 25,
 'billy': 26,
 'worms': 27,
 'he': 28,
 'in': 29,
 'on': 30,
 'like': 31,
 'kids': 32,
 'if': 33,
 'an': 34,
 'at': 35,
 'fried': 36,
 'it': 37,
 'has': 38,
 'you': 39,
 'this': 40,
 'their': 41,
 'only': 42,
 'film': 43,
 'when': 44,
 'me': 45,
 'great': 46,
 'have': 47,
 'can': 48,
 'lot': 49,
 'my': 50,
 'all': 51,
 'wonderful': 52,
 'show': 53,
 'its': 54,
 'about': 55,
 'no': 56,
 'way': 57,
 'out': 58,
 'two': 59,
 'there': 60,
 'not': 61,
 'so': 62,
 'know': 63,
 'children': 64,
 'also': 65,
 'death': 66,
 'liked': 67,
 'some': 68,
 'be': 69,
 'given': 70,
 'or': 71,
 'how': 72,
 'ever': 73,
 'were': 74,
 'any': 75,
 'again': 76,
 'see': 77,
 'loved': 78,
 'brought': 79,
 'mad

In [143]:
len(nlp.stoi)

43773

In [144]:
train_dataset = IMDBDataset(data_dir= DATA_DIR, tokenizer= nlp, train=True)
test_dataset = IMDBDataset(data_dir= DATA_DIR, tokenizer= nlp, train=False)

In [145]:
len(test_dataset)

25000

In [169]:
len(train_dataset[1][0])

201

In [165]:
train_dataset[1][0]

tensor([    1,    40,  1213,    30, 40822,  1965,   509,   589,     4,   570,
          567,   322,    35,     8,  2557, 35167,  2242,    29,   134,    28,
          381,    25,   235,  8245,     3,    35,     8,   177,  1105,    97,
            9,     8,     3,    54,    65,  6067,    16,  1299,     9,   932,
         2972,  2071,    85,   144,  1257,    20,    19,   581,     8,  1802,
          752,     6,   107,    88,    35,     8,    95,    16,    85, 13685,
          642,    20,  9787,   111,  1300,    16,    65,     5,   599, 10223,
           55,    72,    28,     3,    25,  7904,   199,   105,   474,    58,
          125,  1023,    15,     8,   214,   509, 17945,     8,   247,    74,
           61,   130,    81,    37,   138,   918,    11,    25, 10067,    15,
          188,    19,   646,    16,    28,  1084,    17,    47,   192,     4,
         2034,  7813,   445,   101,   242,    97,   361,  6777,   169,    31,
          938, 26959,  4948,  3540,    20,   977,    46,    16, 

In [168]:
data_, label_ = next(iter(test_dataset))
print(data_.size())
print(label_)

torch.Size([200])
tensor([False])


In [192]:
from torch.nn.utils.rnn import pad_sequence

def pad_collate(x):
    data = [item[0] for item in x]
    label = [int(item[1]) for item in x]
    return pad_sequence(data), label

In [193]:
train_dl = DataLoader(train_dataset, batch_size=32, collate_fn= pad_collate)

In [190]:
data_dl_, label_dl_ = next(iter(train_dl))
data_dl_.size()

torch.Size([610, 32])

In [194]:
len(nlp.stoi)

43773

In [196]:
embedding = nn.Embedding(num_embeddings=len(nlp.stoi) ,embedding_dim= 256, padding_idx=0)
embedding(data_dl_).size()

torch.Size([610, 32, 256])

In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim=256, hidden_dim=512, output_dim=, num_layers=2):
        super().__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers= num_layers)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):            # [seq_len, bs]
        x = self.embedding(x)  # [seq_len, bs, 256]
        x = self.lstm(x)       # [seq_len, bs, 512]
        x = self.fc(x)         # [seq_len, ] 





